In [ ]:
!pip install -q pyngrok

In [ ]:
!pip install -q streamlit

In [ ]:
!pip install -q streamlit-ace

In [ ]:
!pip install -q fuzzywuzzy[speedup]

In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
from pyngrok import ngrok

In [ ]:
%%writefile app.py
import streamlit as st
import re
from fuzzywuzzy import process
import json
import requests


def main():

  passportData = requests.get('https://the-icaro.github.io/ago.json')
  namesData = requests.get('https://cspinheiro.github.io/interpol.json')

  def handleBlockedPassports(passportNumber):

    passport = re.findall('[0-9]+', passportNumber)

    blockedData = passportData.json()['ago']

    for data in blockedData:
      for i, v in data.items():
        if str(passport[0]) == str(v):
          return 'Proibido'
    
    return 'Permitido'

  def handleBlockedPersons(name):

    blockedNamesData = namesData.json()['interpol']

    blocked = []

    for data in blockedNamesData:
      for i, v in data.items():
        blocked.append(v)

    searchList = process.extract(str(name), blocked)

    for i in searchList:
      if i[1] >= 85:
        return 'Proibido'

      return 'Permitido'


  def handleImigrate(name, passportNumber):

      passport = handleBlockedPassports(passportNumber)

      person = handleBlockedPersons(name)

      if passport == 'Proibido' or person == 'Proibido':
         st.error("Você não está permitido entrar nesse País")
         return
      
      st.success("Você pode entrar nesse País")
      return

  st.set_page_config(page_title="Aeroporto", page_icon="✈️", layout="centered")
  st.title("📋 Segurança do Aeroporto")
  form = st.form(key="form")

  with form:
    name = st.text_input("* Nome Completo:")
    passport = st.text_input("* Número do Passaporte:")
    submitted = st.form_submit_button(label="Verificar")

  if submitted:
    if not name or not passport:
      st.warning("Preencha todos os Campos!")
    else :
      handleImigrate(name, passport)

if __name__ == '__main__':
  main()

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501